In [1]:
# --- CELDA 1: CARGA Y PREPROCESAMIENTO (DATOS HISTÓRICOS 2015-2023) ---

import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# ==============================================================================
# 1. CARGA DE DATOS
# ==============================================================================
try:
    df_matricula = pd.read_csv('../data/matricula_senescyt_2015_2023.csv')
    df_ofertas = pd.read_csv('../data/encuentra_empleo_ofertas_2.csv')
    df_inec = pd.read_csv('../data/inec_enemdu_salarios.csv')
    print("✅ Las 3 bases de datos han sido cargadas correctamente.")
except FileNotFoundError:
    print("❌ Error: Faltan archivos. Asegúrate de subir: SENESCYT, Ofertas e INEC.")

# ==============================================================================
# 2. PROCESAMIENTO: SENESCYT (Estudiantes)
# ==============================================================================
# Calculamos la "Masa Crítica" histórica (Total acumulado 2015-2023)
# Esto refleja la cantidad real de profesionales en el mercado.
df_estudiantes = df_matricula.groupby('carrera')['num_estudiantes'].sum().reset_index()
df_estudiantes['key'] = df_estudiantes['carrera'].str.lower().str.strip()

# ==============================================================================
# 3. PROCESAMIENTO: ENCUENTRA EMPLEO (Ofertas y Salarios Específicos)
# ==============================================================================
df_ofertas['salario_oferta'] = (df_ofertas['salario_minimo'] + df_ofertas['salario_maximo']) / 2
df_ofertas_agg = df_ofertas.groupby('carrera_requerida').agg({
    'titulo_puesto': 'count',
    'salario_oferta': 'mean'
}).reset_index()
df_ofertas_agg.rename(columns={'titulo_puesto': 'num_ofertas'}, inplace=True)
df_ofertas_agg['key'] = df_ofertas_agg['carrera_requerida'].str.lower().str.strip()

# ==============================================================================
# 4. PROCESAMIENTO: INEC (Contexto Macroeconómico)
# ==============================================================================
# Filtramos para usar datos recientes y de nivel universitario
df_inec_clean = df_inec[df_inec['nivel_educacion'] == 'Educación Superior Universitaria'].copy()
# Tomamos el promedio de los últimos registros por sector
df_inec_agg = df_inec_clean.groupby('sector_economico').agg({
    'tasa_empleo_formal': 'mean',
    'tasa_desempleo': 'mean',
    'salario_promedio_mensual': 'mean' # Este es el salario referencial del sector
}).reset_index()
df_inec_agg.rename(columns={'salario_promedio_mensual': 'salario_ref_inec'}, inplace=True)

# ------------------------------------------------------------------------------
# FUNCIÓN DE MAPEO (CARRERA -> SECTOR ECONÓMICO)
# ------------------------------------------------------------------------------
# Esta función es CLAVE para relacionar las carreras con la base del INEC
def mapear_sector(carrera):
    c = carrera.lower()
    if 'sistemas' in c or 'software' in c or 'comunicación' in c or 'informática' in c:
        return 'Información y Comunicación'
    elif 'administración' in c or 'contabilidad' in c or 'economía' in c or 'financ' in c or 'marketing' in c:
        return 'Actividades Financieras' # O Actividades Administrativas
    elif 'medicina' in c or 'enfermería' in c or 'salud' in c or 'odont' in c or 'psicol' in c:
        return 'Salud Humana'
    elif 'civil' in c or 'arquitectura' in c:
        return 'Construcción'
    elif 'mecánica' in c or 'industrial' in c or 'eléctrica' in c:
        return 'Industrias Manufactureras'
    elif 'educación' in c or 'docencia' in c:
        return 'Educación'
    elif 'agro' in c or 'agrícola' in c or 'veterinaria' in c:
        return 'Agricultura y Ganadería'
    elif 'turismo' in c or 'hotelería' in c:
        return 'Alojamiento y Servicios de Comida'
    else:
        return 'Actividades Profesionales y Técnicas' # Default para otras ingenierías/técnicas

# Aplicamos el mapeo a los estudiantes para poder cruzar con INEC
df_estudiantes['sector_economico'] = df_estudiantes['carrera'].apply(mapear_sector)

# ==============================================================================
# 5. UNIFICACIÓN DE DATOS (MERGE TOTAL)
# ==============================================================================
# Paso A: Estudiantes + Ofertas
df_master = pd.merge(df_estudiantes, df_ofertas_agg, on='key', how='left')

# Paso B: + Datos INEC (Usando el sector económico como llave)
df_master = pd.merge(df_master, df_inec_agg, on='sector_economico', how='left')

# Limpieza Final
df_master['num_ofertas'] = df_master['num_ofertas'].fillna(0)
df_master['salario_oferta'] = df_master['salario_oferta'].fillna(df_master['salario_ref_inec']) # Si no hay ofertas, usamos ref INEC
df_master = df_master[['carrera', 'num_estudiantes', 'num_ofertas', 'salario_oferta', 'tasa_empleo_formal', 'tasa_desempleo', 'sector_economico']]

✅ Las 3 bases de datos han sido cargadas correctamente.


In [5]:
# ==============================================================================
# 6. CLUSTERING (K-MEANS MEJORADO)
# ==============================================================================
features = ['num_estudiantes', 'num_ofertas', 'salario_oferta', 'tasa_empleo_formal']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_master[features])

kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_master['cluster'] = kmeans.fit_predict(X_scaled)

# ==============================================================================
# 7. ETIQUETADO Y VISUALIZACIÓN
# ==============================================================================
centroides = df_master.groupby('cluster')[features].mean()
medias = df_master[features].mean()

def etiqueta_inteligente(row, m):
    # Lógica Multivariable
    alta_demanda = row['num_ofertas'] > m['num_ofertas']
    buen_sueldo = row['salario_oferta'] > m['salario_oferta']
    empleo_estable = row['tasa_empleo_formal'] > m['tasa_empleo_formal']
    muchos_graduados = row['num_estudiantes'] > m['num_estudiantes']

    if alta_demanda and buen_sueldo:
        return "En Demanda"
    elif muchos_graduados and (not alta_demanda or not empleo_estable):
        return "Saturadas"
    elif not muchos_graduados and (buen_sueldo or empleo_estable):
        return "Nicho"
    else:
        return "Balanceadas"

mapa_labels = {i: etiqueta_inteligente(centroides.loc[i], medias) for i in centroides.index}
df_master['categoria'] = df_master['cluster'].map(mapa_labels)

# GRÁFICO 3D
fig = px.scatter_3d(
    df_master,
    x='num_estudiantes',
    y='num_ofertas',
    z='salario_oferta',
    color='categoria',
    symbol='categoria',
    hover_name='carrera',
    hover_data={'tasa_empleo_formal':':.1f%', 'sector_economico':True},
    title='<b>Clustering de Carreras 2025</b>',
    labels={
        'num_estudiantes': 'Total Graduados (2015-2023)',
        'num_ofertas': 'Ofertas Activas',
        'salario_oferta': 'Salario Promedio ($)'
    },
    color_discrete_map={
        "En Demanda": "#00CC96",
        "Saturadas": "#EF553B",
        "Nicho": "#636EFA",
        "Balanceadas": "#FECB52"
    },
    opacity=0.9
)

# Ajuste de etiquetas pequeñas
fig.update_layout(
    scene=dict(
        xaxis=dict(title=dict(font=dict(size=10)), tickfont=dict(size=9)),
        yaxis=dict(title=dict(font=dict(size=10)), tickfont=dict(size=9)),
        zaxis=dict(title=dict(font=dict(size=10)), tickfont=dict(size=9)),
    ),
    margin=dict(l=0, r=0, b=0, t=40),
    legend=dict(title="Categoría", font=dict(size=10))
)

fig.show()

d:\Anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.

